<a href="https://colab.research.google.com/github/muesgit/deep_learning_tutorial/blob/main/FishVsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Build Training Dataset
import torchvision
from torchvision import transforms

train_data_path = '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/train'
# to secure the model from overfitting, the validation data set is created
# after each training cycle (epoch) it is compared against it
val_data_path = '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/val'
# test model after all training has been completed
test_data_path = '/content/drive/MyDrive/Colab Notebooks/Pytorch_Book/test'

transforms = transforms.Compose([
    # Scale incoming image to defined Resolution
    transforms.Resize(64),
    # Convert image to tensor
    transforms.ToTensor(),
    # normalize image to keep the incoming values between 0 and 1
    # (exploding gradient problem), Values from ImageNet dataset
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data  = torchvision.datasets.ImageFolder(root=train_data_path,
                                              transform=transforms)
val_data    = torchvision.datasets.ImageFolder(root=val_data_path,
                                            transform=transforms)
test_data   = torchvision.datasets.ImageFolder(root=test_data_path,
                                             transform=transforms)

In [8]:
import torch.utils.data
from torch.utils.data import DataLoader
# Build Loader
# Batch Size (mini-batches require less memory)
batch_size = 64
train_data_loader = DataLoader(train_data, batch_size=batch_size)
val_data_loader   = DataLoader(val_data, batch_size=batch_size)
test_data_loader  = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Building fully connected neural network
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # the fully connected layers, numbers in between are arbitraty
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        # last layer has size of classes (two: Cat, Fish)
        self.fc3 = nn.Linear(50, 2)
    def forward(self, x):
        # transform x (datashape)
        x = x.view(-1, 12288)
        # relu, rectified linear output
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # since CrossEntropyLoss function is used for Loss determination,
        # no softmax is used for the last layer
        x = self.fc3(x)
        return x

simplenet = SimpleNet()